In [0]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from sparkmeasure import StageMetrics
from pyspark.sql import functions as F
from operator import add
import operator
from pyspark.sql.functions import broadcast
from pyspark.sql.types import StringType
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType
from operator import add
import re

spark = SparkSession.builder.appName("task3").getOrCreate()

stagemetrics = StageMetrics(spark)
dbfs_fileStore_prefix = "/FileStore/tables"
prefix = "ontimeperformance"
size = "small"

In [0]:
# https://stackoverflow.com/questions/55453101/pyspark-error-analysisexception-cannot-resolve-column-name
def clean_column_names(df):
  tempList = [] #Edit01
  for col in df.columns:
      new_name = col.strip()
      new_name = "".join(new_name.split())
      new_name = new_name.replace('.','') 
      tempList.append(new_name) 

  return df.toDF(*tempList) 

In [0]:
def print_models_df(airlines):
    if len(airlines) == 0: return
    i = 0
    curr = airlines[0][0]
    has_print = False
    aircraft_type = ''
    while i < len(airlines):
        if curr == airlines[i][0]:
            aircraft_type += airlines[i][1] + " " + airlines[i][2] + ", "
            i += 1
        else:
            has_print = True
            print(curr + " \t " +  "[" + aircraft_type[:-2] + "]")
            curr = airlines[i][0]
            aircraft_type = ""
    if not has_print: print(curr + " \t " + "[" + aircraft_type[:-2] + "]")

def task_3_df(spark_session, flights_path, airlines_path, aircrafts_path, country):
    flights_df = spark.read.format("csv") \
                        .option("header", "true") \
                        .option("inferSchema", "true") \
                        .load(flights_path)
    airlines_df = spark.read.format("csv") \
                        .option("header", "true") \
                        .option("inferSchema", "true") \
                        .load(airlines_path)
    aircrafts_df = spark.read.format("csv") \
                        .option("header", "true") \
                        .option("inferSchema", "true") \
                        .load(aircrafts_path)
    # Clean
    flights_df = clean_column_names(flights_df)
    airlines_df = clean_column_names(airlines_df)
    aircraft_df = clean_column_names(aircrafts_df)
    
    # Join the Airline and Flight datasets
    airlines_count = (airlines_df.join(flights_df, airlines_df.carrier_code == flights_df.carrier_code)
                      .drop(airlines_df.carrier_code).filter(F.col("country") == country)
                      .groupBy("carrier_code", "name", "country", "tail_number").count())
    
    # Format the model name
    aircraft_model = (aircraft_df.withColumn("model_name", F.regexp_extract(F.col("model"), "[A-Z]*-?\d{1,3}", 0)))
    
    # Join the airline count and aircraft datasets
    airlines_count = (airlines_count.join(aircraft_model, F.broadcast(airlines_count).tail_number == aircraft_model.tailnum, how="inner")
                      .groupBy("name", "manufacturer", "model_name").sum("count"))
    
    # Filter for NULL
    airlines_count = airlines_count.filter(F.col("model_name").isNotNull()).filter(F.col("manufacturer").isNotNull())
    
    # Define window_spec to orderby sum of count
    window_spec = Window.partitionBy(airlines_count["name"]).orderBy(airlines_count["sum(count)"].desc())
    
    # Rank the airlines
    airlines = (airlines_count.select("*", F.rank().over(window_spec).alias("aircraft_rank"))
                .filter(F.col("aircraft_rank") < 6).sort(["name", "aircraft_rank"]))
    
    # Print output
    print_models_df(airlines.collect())


In [0]:
stagemetrics.begin()

task_3_df(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_small.csv", 
                                f"{dbfs_fileStore_prefix}/{prefix}_airlines.csv", 
                                f"{dbfs_fileStore_prefix}/{prefix}_aircrafts.csv", "United States")
stagemetrics.end()

AirTran [BOEING 717, BOEING 737]
Alaska Airlines Inc. [BOEING 737, MCDONNELL DOUGLAS DC-9, MCDONNELL DOUGLAS AIRCRAFT CO 83]
American Airlines Inc. [MCDONNELL DOUGLAS DC-9, AIRBUS INDUSTRIE A319, BOEING 757, BOEING 767, AIRBUS INDUSTRIE A320]
American Eagle Airlines Inc. [EMBRAER EMB-145, EMBRAER EMB-135, BOMBARDIER INC CL-600, SAAB-SCANIA 340]
Atlantic Southeast Airlines [BOMBARDIER INC CL-600, EMBRAER EMB-145, CANADAIR CL-600, EMBRAER EMB-135, AEROSPATIALE/ALENIA ATR-72]
Comair [BOMBARDIER INC CL-600, CANADAIR CL-600, BOMBARDIER INC CL600, PIPER PA-28]
Continental Air Lines Inc. [BOEING 737, BOEING 757, BOEING 767, BOEING 777]
Delta Air Lines Inc. [MCDONNELL DOUGLAS AIRCRAFT CO MD-88, BOEING 757, BOEING 737, BOEING 717, BOEING 767]
Frontier Airlines Inc. [AIRBUS A319, AIRBUS A318, AIRBUS INDUSTRIE A319, AIRBUS A320]
Hawaiian Airlines Inc. [BOEING 717, BOEING 767]
Independence Air [BOMBARDIER INC CL-600]
JetBlue Airways [AIRBUS A320, EMBRAER 190, AIRBUS INDUSTRIE A320]
JetSuite Air [EMBRAER EMB-145, EMBRAER EMB-135]
Mesa Airlines Inc. [BOMBARDIER INC CL600, BOMBARDIER INC CL-600, DEHAVILLAND DHC-8, CANADAIR CL-600]
Northwest Airlines Inc. [MCDONNELL DOUGLAS DC-9, AIRBUS INDUSTRIE A320, BOEING 757, DOUGLAS DC-9, AIRBUS A319]
PSA Airlines Inc. [BOMBARDIER INC CL-600, CANADAIR CL-600, BOMBARDIER INC CL600, PIPER PA-28]
Pinnacle Airlines Inc. [BOMBARDIER INC CL-600, BOMBARDIER INC CL600]
Skywest Airlines Inc. [BOMBARDIER INC CL-600, EMBRAER EMB-120, BOMBARDIER INC CL600, CANADAIR CL-600]
Southwest Airlines Co. [BOEING 737, BOEING 747, EMBRAER EMB-120, CESSNA T337, BEECH 65]
US Airways [BOEING 737, AIRBUS INDUSTRIE A319, AIRBUS INDUSTRIE A320, AIRBUS INDUSTRIE A321, EMBRAER 190]

In [0]:
stagemetrics.begin()

task_3_df(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_medium.csv", 
                                f"{dbfs_fileStore_prefix}/{prefix}_airlines.csv", 
                                f"{dbfs_fileStore_prefix}/{prefix}_aircrafts.csv", "United States")
stagemetrics.end()

AirTran [BOEING 717, BOEING 737]
Alaska Airlines Inc. [BOEING 737, MCDONNELL DOUGLAS DC-9, MCDONNELL DOUGLAS AIRCRAFT CO 83]
American Airlines Inc. [MCDONNELL DOUGLAS DC-9, AIRBUS INDUSTRIE A319, BOEING 757, BOEING 767, AIRBUS INDUSTRIE A320]
American Eagle Airlines Inc. [EMBRAER EMB-145, EMBRAER EMB-135, BOMBARDIER INC CL-600, SAAB-SCANIA 340]
Atlantic Southeast Airlines [BOMBARDIER INC CL-600, EMBRAER EMB-145, CANADAIR CL-600, EMBRAER EMB-135, AEROSPATIALE/ALENIA ATR-72]
Comair [BOMBARDIER INC CL-600, CANADAIR CL-600, BOMBARDIER INC CL600, PIPER PA-28]
Continental Air Lines Inc. [BOEING 737, BOEING 757, BOEING 767, BOEING 777]
Delta Air Lines Inc. [MCDONNELL DOUGLAS AIRCRAFT CO MD-88, BOEING 757, BOEING 737, BOEING 717, AIRBUS INDUSTRIE A320]
Frontier Airlines Inc. [AIRBUS A319, AIRBUS A318, AIRBUS INDUSTRIE A319, AIRBUS A320]
Hawaiian Airlines Inc. [BOEING 717, BOEING 767]
Independence Air [BOMBARDIER INC CL-600]
JetBlue Airways [AIRBUS A320, EMBRAER 190, AIRBUS INDUSTRIE A320]
JetSuite Air [EMBRAER EMB-145, EMBRAER EMB-135]
Mesa Airlines Inc. [BOMBARDIER INC CL600, BOMBARDIER INC CL-600, DEHAVILLAND DHC-8, CANADAIR CL-600]
Northwest Airlines Inc. [MCDONNELL DOUGLAS DC-9, AIRBUS INDUSTRIE A320, BOEING 757, DOUGLAS DC-9, AIRBUS INDUSTRIE A319]
PSA Airlines Inc. [BOMBARDIER INC CL-600, CANADAIR CL-600, BOMBARDIER INC CL600, PIPER PA-28]
Pinnacle Airlines Inc. [BOMBARDIER INC CL-600, BOMBARDIER INC CL600]
Skywest Airlines Inc. [BOMBARDIER INC CL-600, EMBRAER EMB-120, BOMBARDIER INC CL600, CANADAIR CL-600]
Southwest Airlines Co. [BOEING 737, BOEING 747, EMBRAER EMB-120, CESSNA T337, BEECH 65]
US Airways [BOEING 737, AIRBUS INDUSTRIE A319, AIRBUS INDUSTRIE A320, AIRBUS INDUSTRIE A321, EMBRAER 190]

In [0]:
stagemetrics.begin()

task_3_df(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_large.csv", 
                                f"{dbfs_fileStore_prefix}/{prefix}_airlines.csv", 
                                f"{dbfs_fileStore_prefix}/{prefix}_aircrafts.csv", "United States")
stagemetrics.end()

AirTran [BOEING 717, BOEING 737]
Alaska Airlines Inc. [BOEING 737, MCDONNELL DOUGLAS DC-9, MCDONNELL DOUGLAS AIRCRAFT CO 83]
American Airlines Inc. [MCDONNELL DOUGLAS DC-9, AIRBUS INDUSTRIE A319, BOEING 757, BOEING 767, AIRBUS INDUSTRIE A320]
American Eagle Airlines Inc. [EMBRAER EMB-145, EMBRAER EMB-135, BOMBARDIER INC CL-600, SAAB-SCANIA 340]
Atlantic Southeast Airlines [BOMBARDIER INC CL-600, EMBRAER EMB-145, CANADAIR CL-600, EMBRAER EMB-135, AEROSPATIALE/ALENIA ATR-72]
Comair [BOMBARDIER INC CL-600, CANADAIR CL-600, BOMBARDIER INC CL600, PIPER PA-28]
Continental Air Lines Inc. [BOEING 737, BOEING 757, BOEING 767, BOEING 777, EMBRAER EMB-145]
Delta Air Lines Inc. [MCDONNELL DOUGLAS AIRCRAFT CO MD-88, BOEING 757, BOEING 737, BOEING 717, AIRBUS INDUSTRIE A320]
Frontier Airlines Inc. [AIRBUS A319, AIRBUS A318, AIRBUS INDUSTRIE A319, AIRBUS A320]
Hawaiian Airlines Inc. [BOEING 717, BOEING 767]
Independence Air [BOMBARDIER INC CL-600]
JetBlue Airways [AIRBUS A320, EMBRAER 190, AIRBUS INDUSTRIE A320]
JetSuite Air [EMBRAER EMB-145, EMBRAER EMB-135]
Mesa Airlines Inc. [BOMBARDIER INC CL600, BOMBARDIER INC CL-600, DEHAVILLAND DHC-8, CANADAIR CL-600, BOEING OF CANADA LTD DHC-8]
Northwest Airlines Inc. [MCDONNELL DOUGLAS DC-9, AIRBUS INDUSTRIE A320, BOEING 757, DOUGLAS DC-9, AIRBUS A319]
PSA Airlines Inc. [BOMBARDIER INC CL-600, CANADAIR CL-600, BOMBARDIER INC CL600, PIPER PA-28]
Pinnacle Airlines Inc. [BOMBARDIER INC CL-600, BOMBARDIER INC CL600]
Skywest Airlines Inc. [BOMBARDIER INC CL-600, EMBRAER EMB-120, BOMBARDIER INC CL600, CANADAIR CL-600, BOEING 737]
Southwest Airlines Co. [BOEING 737, BOEING 747, EMBRAER EMB-120, CESSNA T337, BEECH 65]
US Airways [BOEING 737, AIRBUS INDUSTRIE A319, AIRBUS INDUSTRIE A320, AIRBUS INDUSTRIE A321, EMBRAER 190]

In [0]:
stagemetrics.begin()

task_3_df(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_massive.csv", 
                                f"{dbfs_fileStore_prefix}/{prefix}_airlines.csv", 
                                f"{dbfs_fileStore_prefix}/{prefix}_aircrafts.csv", "United States")
stagemetrics.end()

AirTran [BOEING 717, BOEING 737]
Alaska Airlines Inc. [BOEING 737, MCDONNELL DOUGLAS DC-9, MCDONNELL DOUGLAS AIRCRAFT CO 83]
American Airlines Inc. [MCDONNELL DOUGLAS DC-9, AIRBUS INDUSTRIE A319, BOEING 757, BOEING 767, AIRBUS INDUSTRIE A320]
American Eagle Airlines Inc. [EMBRAER EMB-145, EMBRAER EMB-135, BOMBARDIER INC CL-600, SAAB-SCANIA 340]
Atlantic Southeast Airlines [BOMBARDIER INC CL-600, EMBRAER EMB-145, CANADAIR CL-600, EMBRAER EMB-135, AEROSPATIALE/ALENIA ATR-72]
Comair [BOMBARDIER INC CL-600, CANADAIR CL-600, BOMBARDIER INC CL600, PIPER PA-28, EMBRAER EMB-145]
Continental Air Lines Inc. [BOEING 737, BOEING 757, BOEING 767, BOEING 777, EMBRAER EMB-145]
Delta Air Lines Inc. [MCDONNELL DOUGLAS AIRCRAFT CO MD-88, BOEING 757, BOEING 737, BOEING 717, AIRBUS INDUSTRIE A320]
Frontier Airlines Inc. [AIRBUS A319, AIRBUS A318, AIRBUS INDUSTRIE A319, AIRBUS A320, BOEING 737]
Hawaiian Airlines Inc. [BOEING 717, BOEING 767]
Independence Air [BOMBARDIER INC CL-600]
JetBlue Airways [AIRBUS A320, EMBRAER 190, AIRBUS INDUSTRIE A320]
JetSuite Air [EMBRAER EMB-145, EMBRAER EMB-135]
Mesa Airlines Inc. [BOMBARDIER INC CL600, BOMBARDIER INC CL-600, DEHAVILLAND DHC-8, CANADAIR CL-600, BOEING OF CANADA LTD DHC-8]
Northwest Airlines Inc. [MCDONNELL DOUGLAS DC-9, AIRBUS INDUSTRIE A320, BOEING 757, DOUGLAS DC-9, AIRBUS A319]
PSA Airlines Inc. [BOMBARDIER INC CL-600, CANADAIR CL-600, BOMBARDIER INC CL600, PIPER PA-28, EMBRAER EMB-145]
Pinnacle Airlines Inc. [BOMBARDIER INC CL-600, BOMBARDIER INC CL600]
Skywest Airlines Inc. [BOMBARDIER INC CL-600, EMBRAER EMB-120, BOMBARDIER INC CL600, CANADAIR CL-600, BOEING 737]
Southwest Airlines Co. [BOEING 737, BOEING 747, EMBRAER EMB-120, CESSNA T337, BEECH 65]
US Airways [BOEING 737, AIRBUS INDUSTRIE A319, AIRBUS INDUSTRIE A320, AIRBUS INDUSTRIE A321, EMBRAER 190]